In [1]:
!pip install "sap-llm-commons[all]"

In [2]:
!pip install ai-api-client-sdk

In [1]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

C:\Users\kylin\AppData\Local\Temp\ipykernel_15092\3646097473.py:5: UserWarning: Starting from verison 1.0.0 the default proxy_version was set to 'gen-ai-hub'. Use gen_ai_hub.proxy.core.proxy_clients.set_proxy_version('btp') to set the proxy_version to 'btp'.
  import llm_commons.proxy.base


In [2]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [3]:
resource_group = widgets.Text(
    value='default', # resource group
    placeholder='Resource group of deployments',
    description='',
    disabled=False
)

resource_group

Text(value='default', placeholder='Resource group of deployments')

In [4]:
with open('config/irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

In [5]:
os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'
os.environ["TAVILY_API_KEY"] = sk['Tavily']

llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']

In [6]:
aic_proxy_client = AICoreProxyClient()
btp_proxy_client = BTPProxyClient()

In [7]:
PDF_NAME="ai.pdf"
from langchain.document_loaders import PyMuPDFLoader
docs = PyMuPDFLoader(PDF_NAME).load()

print (f'There are {len(docs)} document(s) in {PDF_NAME}.')
print (f'There are {len(docs[0].page_content)} characters in the first page of your document.')

There are 86 document(s) in ai.pdf.
There are 117 characters in the first page of your document.


Deploy customized fine-tuned models

In [8]:
AICoreProxyClient.add_foundation_model_scenario(
    scenario_id='fine-tuned-llm',
    config_names='fine-tuned-*',
    prediction_url_suffix='/v1/completions'
)

In [9]:
aic_proxy_client.get_deployments()

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc8e3c266c8881f0', config_id='58a5ffea-1afe-44b4-b6b9-ca749776e58e', config_name='tiiuae--falcon-40b-instruct-config', deployment_id='dc8e3c266c8881f0', model_name='tiiuae--falcon-40b-instruct', created_at=datetime.datetime(2024, 1, 25, 12, 53, 26, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'aicore-opensource', 'model_version': 'null'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dd88f66091ca1982', config_id='a99e09fb-7e88-4a8d-abe8-af7f8b146cb8', config_name='text-embedding-ada-002-config', deployment_id='dd88f66091ca1982', model_name='text-embedding-ada-002', created_at=datetime.datetime(2024, 1, 25, 12, 16, 20, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'azure-openai', 'model_version': '2'}, custom_prediction_suffix=None),
 Deployment(url='https

In [11]:
#text_davinci_003 = Completion(deployment_id='text-davinci-003')

In [ ]:
#text_davinci_003.create(model_name='our-awesome-model/v1', prompt="San Francisco is a")

Initialize harmonized model

In [10]:
llm = init_llm('gpt-35-turbo', proxy_client = aic_proxy_client,temperature=0., max_tokens=256, deployment_id='d10917558b77a3db', api_base=llm_commons.proxy.api_base)

In [11]:
llm.predict("请打印秋夜将晓出篱门迎凉有感这首诗")

j:\python3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'秋夜将晓出篱门，迎凉有感\n\n秋夜将晓，我走出篱门，\n迎接着凉爽的秋风。\n月亮渐渐西沉，星星点点闪烁，\n夜空中弥漫着宁静的气息。\n\n凉风拂过脸颊，带来一丝清凉，\n让我感受到秋天的美妙。\n树叶沙沙作响，落叶纷纷飘落，\n仿佛是大自然在为秋天歌唱。\n\n夜色渐渐深沉，寂静中有一丝寒意，\n但我却感到心中温暖。\n秋天是收获的季节，是丰收的时刻，\n我期待着秋天带来的丰盛。\n\n秋夜将晓，我'

In [12]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

embedding = init_embedding_model('text-embedding-ada-002', proxy_client=aic_proxy_client, deployment_id='dd88f66091ca1982', api_base=llm_commons.proxy.api_base)
vectorstore = Chroma.from_documents(split_docs, embedding, collection_name="AI.pdf.guide")

j:\python3.12\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [13]:
query = "what is the difference btw sap ai and azure ai?"
similar_docs = vectorstore.similarity_search(query, 3)
similar_docs

[Document(page_content='2 \nService Plans\nThe SAP AI Core service plan you choose determines pricing, conditions of use, resources, available services,\nand hosts. The generative AI hub in SAP AI Core is available only through the sap-internal service plan.\n\ue201\xa0Caution\nThe sap-internal service plan is available only for internal consumption and on eu10 canary.\nThe sap-internal service plan enhances the capabilities provided in the standard service plan. Specifically,\nit provides access to the foundation-models global AI scenario. This scenario, which is managed by SAP AI\nCore, includes serving templates for deployments with integrated LLM access.\nIf you’re new to SAP AI Core, choose the sap-internal service plan during your initial setup. For more\ninformation, see Add the Internal Plan for LLMs to the Global Account [page 8].\nIf you already have an SAP AI Core tenant on a standard or free tier service plan, you can update the service', metadata={'author': '', 'creationDa

In [17]:
query = "What is the world's best B2B company?"
query_embedding = embedding.embed_query(query)

In [21]:
documents = ["This is a sample document"]
document_embedding = embedding.embed_documents(documents)

AI Core proxy support

In [14]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy
from llm_commons.langchain.proxy import ChatOpenAI
chat = ChatOpenAI(proxy_model_name='gpt-4')
chat.predict("What is the world's best contry?")

'As an AI, I don\'t have personal opinions. However, what is considered the "best" country can vary greatly depending on the criteria used to evaluate it. Some might use wealth (GDP), education, healthcare, happiness index, environmental standards, or personal freedoms as determining factors. For example, according to the United Nations\' Human Development Report, Norway ranks highest in terms of human development, which takes into account factors like economic prosperity, education, and health. Meanwhile, the World Happiness Report often ranks Finland as the happiest country in the world. It\'s subjective and depends on personal preferences and needs.'

Proxy client - use proxies in parallel

In [15]:
from llm_commons.langchain.proxy import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage
from llm_commons.proxy.openai import ChatCompletion

In [17]:
messages=[{"role": "user", "content": "Say this is a test"}]

print(ChatCompletion.create(proxy_client=aic_proxy_client, model_name='gpt-4', messages=messages, api_base=os.environ['AICORE_LLM_API_BASE']))

AttributeError: 'NoneType' object has no attribute 'create'

In [18]:
messages = [
    HumanMessage(content="Say this is a test"),
]
# btp_llm = ChatOpenAI(proxy_client=btp_proxy_client, deployment_id='gpt-4')
# print(btp_llm.invoke(messages))
aic_llm = ChatOpenAI(proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
print(aic_llm.invoke(messages))

content='This is a test.'


In [19]:
chat = ChatOpenAI(temperature=0,proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
response = chat([ HumanMessage(content="Hello Langchain! can you give me some self introduction?") ])
print(response)


j:\python3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content="Hello! I'm Langchain, an AI developed by OpenAI. I'm designed to assist with a wide range of tasks, including answering questions, providing information, and helping with various tasks. I'm constantly learning and improving, so I can provide the most accurate and helpful responses. I'm here to help you, so feel free to ask me anything!"


In [20]:
response = chat.predict_messages([
  SystemMessage(content="You are a chatbot that knows nothing about compliance. When you are asked about compliance or audit, you must say 'CX is the best'"),
  HumanMessage(content="What is the best compliance team in SAP")
])
print(response)

j:\python3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='CX is the best.'


test a search api

In [30]:
os.environ["TAVILY_API_KEY"] = "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"
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "taylor swift ai pic"})

[{'url': 'https://www.npr.org/2024/01/26/1227091070/deepfakes-taylor-swift-images-regulation',
  'content': 'Deepfakes pose a growing challenge, with little regulation\nThe risk of damage from deepfakes is far-ranging, from the appropriation of women\'s faces to make explicit sex videos, to the use of celebrities in unapproved promotions, to the use of manipulated images in political disinformation campaigns.\n Technology\nDeepfakes exploiting Taylor Swift images exemplify a scourge with little oversight\nBy\nBill Chappell\nA photo illustration created last July shows an advertisement to create AI girls reflected in a public service announcement issued by the FBI regarding malicious actors manipulating photos and videos to create explicit content and sextortion schemes. Stefani Reynolds/AFP via Getty Images\nhide caption\nA photo illustration created last July shows an advertisement to create AI girls reflected in a public service announcement issued by the FBI regarding malicious acto

Using llm-commons with fine-tuned models

In [23]:
# use it with langchain

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from llm_commons.langchain.proxy import OpenAI

llm = ChatOpenAI(proxy_model_name='gpt-4')
template = "Question: {question}\nAnswer: Let's think step by step."""

In [24]:
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
llm_chain.run("What is the world's best B2B company?")

j:\python3.12\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Question: What is the world's best B2B company?
Answer: Let's think step by step.

> Finished chain.


'The term "best" is subjective and can vary based on different criteria such as revenue, customer satisfaction, innovation, market share, etc. However, some of the world\'s most successful B2B companies include Microsoft, IBM, and Alibaba. Microsoft is a leading player in providing software, hardware, and services to businesses of all sizes. IBM is known for its IT services and consulting. Alibaba\'s B2B platform connects suppliers with buyers worldwide. All these companies have made significant impacts in the B2B sector.'

In [25]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()

search.run("what day is it today?")



DuckDuckGoSearchException: _aget_url() https://duckduckgo.com RequestsError: Failed to perform, ErrCode: 28, Reason: 'Failed to connect to duckduckgo.com port 443 after 21240 ms: Couldn't connect to server'. This may be a libcurl error, See https://curl.se/libcurl/c/libcurl-errors.html first for more details.